# MRI-QC
https://mriqc.readthedocs.io/en/latest/docker.html#docker

## Test

In [ ]:
docker run -it nipreps/mriqc:latest --version

## Run the participant level in subjects 001 002 003

If the argument `--participant_label` is not provided, then all subjects will be processed and the group level analysis will automatically be executed without need of running the command in item 3.

Paths `<bids_dir>` and `<output_dir>` must be absolute. In particular, specifying relative paths for `<output_dir>` will generate no error and mriqc will run to completion without error but produce no output.

For security reasons, we recommend to run the docker command with the options `--read-only --tmpfs /run --tmpfs /tmp`. This will run the docker image in read-only mode, and map the temporary folders `/run` and `/tmp` to the temporal folder of the host.

In [ ]:
# default
docker run -it --rm -v <bids_dir>:/data:ro -v <output_dir>:/out nipreps/mriqc:latest /data /out participant --participant_label 001 002 003
# with my paths
docker run -it --rm -v /home/jaimebarranco/Desktop/samples_v3_bids:/data:ro -v /home/jaimebarranco/Desktop/mriqc_output:/out nipreps/mriqc:latest /data /out participant --participant_label 001 002 003
# handle performance
docker run -it --rm -v /home/jaimebarranco/Desktop/samples_v3_bids:/data:ro -v /home/jaimebarranco/Desktop/mriqc_output:/out nipreps/mriqc:latest /data /out participant --participant_label 001 --nprocs 12 --omp-nthreads 12

## Run the group level and report generation on previously processed (use the same `<output_dir>`) subjects

In [ ]:
# default
docker run -it --rm -v <bids_dir>:/data:ro -v <output_dir>:/out nipreps/mriqc:latest /data /out group #--read-only --tmpfs /run --tmpfs /tmp
# with my paths
docker run -it --rm -v /home/jaimebarranco/Desktop/samples_v2_bids:/data:ro -v /home/jaimebarranco/Desktop/mriqc_output:/out nipreps/mriqc:latest /data /out group #--read-only --tmpfs /run --tmpfs /tmp

## Loop of individual participants
And group report

In [ ]:
import os, subprocess

# def run_command(command):
#     process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     console_output, console_errors = process.communicate()

def run_command(command):
    os.system(command)

input_folder = '/home/jaimebarranco/Desktop/samples_v3_bids'
output_folder = '/home/jaimebarranco/Desktop/mriqc_output'
n_procs = 12
n_threads = 12

n_sub = [f.path for f in os.scandir(input_folder) if f.is_dir() and f.name.startswith('sub-')] # number of subjects in input folder
n_sub = len(n_sub)
print(f'Number of subjects: {n_sub}')

# Run MRIQC for each subject
for sub in range(6, n_sub):
    command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out nipreps/mriqc:latest /data /out participant --participant_label {sub+1:03} --nprocs {n_procs} --omp-nthreads {n_threads}'
    print(command)
    run_command(command)

# Run MRIQC group analysis
command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out nipreps/mriqc:latest /data /out group'
print(command)
run_command(command)

## Metrics correlation
Meri's subjective scores vs IQMs (Image Quality Metrics)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# read an excel file in .xlsx format
df = pd.read_excel('/home/jaimebarranco/Desktop/scores.xlsx', sheet_name='samples_v3')

# metrics
meri = df['quality'].to_numpy()
cjv = df['cjv'].to_numpy()
cnr = df['cnr'].to_numpy()
efc = df['efc'].to_numpy()
fber = df['fber'].to_numpy()
fwhm_avg = df['fwhm_avg'].to_numpy()
fwhm_x = df['fwhm_x'].to_numpy()
fwhm_y = df['fwhm_y'].to_numpy()
fwhm_z = df['fwhm_z'].to_numpy()
icvs_csf = df['icvs_csf'].to_numpy()
icvs_gm = df['icvs_gm'].to_numpy()
icvs_wm = df['icvs_wm'].to_numpy()
inu_med = df['inu_med'].to_numpy()
inu_range = df['inu_range'].to_numpy()
qi_1 = df['qi_1'].to_numpy()
qi_2 = df['qi_2'].to_numpy()
rpve_csf = df['rpve_csf'].to_numpy()
rpve_gm = df['rpve_gm'].to_numpy()
rpve_wm = df['rpve_wm'].to_numpy()
snr_csf = df['snr_csf'].to_numpy()
snr_gm = df['snr_gm'].to_numpy()
snr_total = df['snr_total'].to_numpy()
snr_wm = df['snr_wm'].to_numpy()
snrd_csf = df['snrd_csf'].to_numpy()
snrd_gm = df['snrd_gm'].to_numpy()
snrd_total = df['snrd_total'].to_numpy()
snrd_wm = df['snrd_wm'].to_numpy()
summary_bg_k = df['summary_bg_k'].to_numpy()
summary_bg_mad = df['summary_bg_mad'].to_numpy()
summary_bg_mean = df['summary_bg_mean'].to_numpy()
summary_bg_median = df['summary_bg_median'].to_numpy()
summary_bg_n = df['summary_bg_n'].to_numpy()
summary_bg_p05 = df['summary_bg_p05'].to_numpy()
summary_bg_p95 = df['summary_bg_p95'].to_numpy()
summary_bg_stdv = df['summary_bg_stdv'].to_numpy()
summary_csf_k = df['summary_csf_k'].to_numpy()
summary_csf_mad = df['summary_csf_mad'].to_numpy()
summary_csf_mean = df['summary_csf_mean'].to_numpy()
summary_csf_median = df['summary_csf_median'].to_numpy()
summary_csf_n = df['summary_csf_n'].to_numpy()
summary_csf_p05 = df['summary_csf_p05'].to_numpy()
summary_csf_p95 = df['summary_csf_p95'].to_numpy()
summary_csf_stdv = df['summary_csf_stdv'].to_numpy()
summary_gm_k = df['summary_gm_k'].to_numpy()
summary_gm_mad = df['summary_gm_mad'].to_numpy()
summary_gm_mean = df['summary_gm_mean'].to_numpy()
summary_gm_median = df['summary_gm_median'].to_numpy()
summary_gm_n = df['summary_gm_n'].to_numpy()
summary_gm_p05 = df['summary_gm_p05'].to_numpy()
summary_gm_p95 = df['summary_gm_p95'].to_numpy()
summary_gm_stdv = df['summary_gm_stdv'].to_numpy()
summary_wm_k = df['summary_wm_k'].to_numpy()
summary_wm_mad = df['summary_wm_mad'].to_numpy()
summary_wm_mean = df['summary_wm_mean'].to_numpy()
summary_wm_median = df['summary_wm_median'].to_numpy()
summary_wm_n = df['summary_wm_n'].to_numpy()
summary_wm_p05 = df['summary_wm_p05'].to_numpy()
summary_wm_p95 = df['summary_wm_p95'].to_numpy()
summary_wm_stdv = df['summary_wm_stdv'].to_numpy()
tpm_overlap_csf = df['tpm_overlap_csf'].to_numpy()
tpm_overlap_gm = df['tpm_overlap_gm'].to_numpy()
tpm_overlap_wm = df['tpm_overlap_wm'].to_numpy()
wm2max = df['wm2max'].to_numpy()

# add all the metrics above to a list
metrics = [meri, cjv, cnr, efc, fber, fwhm_avg, fwhm_x, fwhm_y, fwhm_z, icvs_csf, icvs_gm, icvs_wm, inu_med, inu_range, qi_1, qi_2, rpve_csf, rpve_gm, rpve_wm, snr_csf, snr_gm, snr_total, snr_wm, snrd_csf, snrd_gm, snrd_total, snrd_wm, summary_bg_k, summary_bg_mad, summary_bg_mean, summary_bg_median, summary_bg_n, summary_bg_p05, summary_bg_p95, summary_bg_stdv, summary_csf_k, summary_csf_mad, summary_csf_mean, summary_csf_median, summary_csf_n, summary_csf_p05, summary_csf_p95, summary_csf_stdv, summary_gm_k, summary_gm_mad, summary_gm_mean, summary_gm_median, summary_gm_n, summary_gm_p05, summary_gm_p95, summary_gm_stdv, summary_wm_k, summary_wm_mad, summary_wm_mean, summary_wm_median, summary_wm_n, summary_wm_p05, summary_wm_p95, summary_wm_stdv, tpm_overlap_csf, tpm_overlap_gm, tpm_overlap_wm, wm2max]
metrics_name = [i for i in df.columns if i not in ['subject', 'sub number', 'comments', 'Sex', 'Age', 'Height', 'Weight', 'BMI', 'axial_length']]

ALPHA = 0.05

# correlation between Meri and all the metrics
for i in range(len(metrics)):
    # R-squared measures the proportion of the variance in one variable that is predictable from the other variable.
    # p_value is a measure of the evidence against a null hypothesis. A small p-value indicates strong evidence against the null hypothesis, while a large p-value indicates weak evidence against the null hypothesis 
    r_squared, p_value = stats.pearsonr(meri, metrics[i])
    if p_value < ALPHA:
        print(f'{metrics_name[i]}: r^2 = {r_squared:.4}, p-value = {p_value:.4}')

# plot Meri vs. all the metrics
# for i in range(len(metrics)):
#     plt.figure(figsize=(10, 10))
#     plt.scatter(meri, metrics[i])
#     plt.xlabel('Meri')
#     plt.ylabel(metrics_name[i])
#     plt.show()


# Fetal Brain QC (Thomas)

In [ ]:
# usage: qc_list_bids_csv [-h] [--mask-patterns MASK_PATTERNS [MASK_PATTERNS ...]] [--out-csv OUT_CSV] [--anonymize-name | --no-anonymize-name] bids-dir
qc_list_bids_csv --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "sub-{subject}_mask.nii.gz" --out-csv /home/jaimebarranco/Desktop/fetal_qc_output/bids_csv.csv /home/jaimebarranco/Desktop/samples_v3_bids
qc_list_bids_csv --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "sub-{subject}_mask.nii.gz" --out-csv /home/jaimebarranco/Desktop/fetal_qc_output/bids_csv.csv /home/jaimebarranco/Desktop/samples_v3_bids

In [ ]:
qc_run_pipeline --bids_dir /home/jaimebarranco/Desktop/samples_v3_bids --out_path /home/jaimebarranco/Desktop/fetal_qc_output/ --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "_mask.nii.gz"
qc_run_pipeline --bids_dir /home/jaimebarranco/Desktop/samples_v3_bids --out_path /home/jaimebarranco/Desktop/fetal_qc_output/

In [ ]:
# Thomas commands - branch mreye
# being in the data folder
qc_list_bids_csv . --mask-patterns-base derivatives/masks/ --out-csv bids.csv --mask-patterns "sub-{subject}_mask.nii.gz" --suffix T1w # index the folder
qc_generate_reports derivatives/reports bids.csv # generate the reports
# mine
qc_list_bids_csv /home/jaimebarranco/Desktop/samples_v3_bids --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "sub-{subject}_mask.nii.gz" --suffix T1w --out-csv /home/jaimebarranco/Desktop/fetal_qc_output/bids_csv.csv
qc_generate_reports /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/reports /home/jaimebarranco/Desktop/fetal_qc_output/bids_csv.csv
# pipeline
qc_run_pipeline --bids_dir /home/jaimebarranco/Desktop/samples_v3_bids --out_path /home/jaimebarranco/Desktop/fetal_qc_output/ --mask-patterns-base /home/jaimebarranco/Desktop/samples_v3_bids/derivatives/masks/ --mask-patterns "sub-{subject}_mask.nii.gz" --suffix T1w

# MREye-QC

In [ ]:
# mreye-qc
docker run -it --rm -v /home/jaimebarranco/Desktop/samples_v3_bids:/data:ro -v /home/jaimebarranco/Desktop/mreyeqc_output:/out mreyeqc:latest /data /out participant --participant_label 001 --nprocs 12 --omp-nthreads 12
# 7bd99c588704826d399898ec1f7419d40dc09b27473d511066f3cf8ab097fe5a

Loop

In [1]:
import os, subprocess

# def run_command(command):
#     process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
#     console_output, console_errors = process.communicate()

def run_command(command):
    os.system(command)

input_folder = '/home/jaimebarranco/Desktop/samples_v3_bids'
output_folder = '/home/jaimebarranco/Desktop/mreyeqc_output'
n_procs = 12
n_threads = 12

n_sub = [f.path for f in os.scandir(input_folder) if f.is_dir() and f.name.startswith('sub-')] # number of subjects in input folder
n_sub = len(n_sub)
print(f'Number of subjects: {n_sub}')

# Run MRIQC for each subject
for sub in range(6, n_sub):
    command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out mreyeqc:latest /data /out participant --participant_label {sub+1:03} --nprocs {n_procs} --omp-nthreads {n_threads}'
    print(command)
    run_command(command)

# Run MRIQC group analysis
command = f'docker run --rm -v {input_folder}:/data:ro -v {output_folder}:/out mreyeqc:latest /data /out group'
print(command)
run_command(command)

Number of subjects: 83
docker run --rm -v /home/jaimebarranco/Desktop/samples_v3_bids:/data:ro -v /home/jaimebarranco/Desktop/mreyeqc_output:/out mreyeqc:latest /data /out participant --participant_label 007 --nprocs 12 --omp-nthreads 12


You are using MRIQC v0.1.dev3288+g5f95281.d20230628, and a newer version is available: 23.1.0.


230629-18:47:34,443 cli IMPORTANT:
	 
    Running MRIQC version 0.1.dev3288+g5f95281.d20230628:
      * BIDS dataset path: /data.
      * Output folder: /out.
      * Analysis levels: ['participant'].

230629-18:47:40,697 cli WARNING:
	 IMPORTANT: Anonymized quality metrics (IQMs) will be submitted to MRIQC's metrics repository. Submission of IQMs can be disabled using the ``--no-sub`` argument. Please visit https://mriqc.readthedocs.io/en/latest/dsa.html to revise MRIQC's Data Sharing Agreement.
230629-18:50:14,54 nipype.interface WARNING:
	 QC metrics upload failed to create an ID for the record uplOADED. rEsponse from server follows: {"_status": "ERR", "_error": {"code": 401, "message": "Please provide proper credentials"}}
230629-18:50:14,54 nipype.interface WARNING:
	 QC metrics failed to upload. Status 401: {"_status": "ERR", "_error": {"code": 401, "message": "Please provide proper credentials"}}
230629-18:50:17,11 cli WARNING:
	 IMPORTANT: Anonymized quality metrics (IQMs) will

You are using MRIQC v0.1.dev3288+g5f95281.d20230628, and a newer version is available: 23.1.0.


230629-18:50:22,85 cli IMPORTANT:
	 
    Running MRIQC version 0.1.dev3288+g5f95281.d20230628:
      * BIDS dataset path: /data.
      * Output folder: /out.
      * Analysis levels: ['participant'].

230629-18:50:28,352 cli WARNING:
	 IMPORTANT: Anonymized quality metrics (IQMs) will be submitted to MRIQC's metrics repository. Submission of IQMs can be disabled using the ``--no-sub`` argument. Please visit https://mriqc.readthedocs.io/en/latest/dsa.html to revise MRIQC's Data Sharing Agreement.
230629-18:52:45,116 nipype.interface WARNING:
	 QC metrics upload failed to create an ID for the record uplOADED. rEsponse from server follows: {"_status": "ERR", "_error": {"code": 401, "message": "Please provide proper credentials"}}
230629-18:52:45,116 nipype.interface WARNING:
	 QC metrics failed to upload. Status 401: {"_status": "ERR", "_error": {"code": 401, "message": "Please provide proper credentials"}}
230629-18:52:48,648 cli WARNING:
	 IMPORTANT: Anonymized quality metrics (IQMs) wi

...

You are using MRIQC v0.1.dev3288+g5f95281.d20230628, and a newer version is available: 23.1.0.


230629-22:00:51,986 cli IMPORTANT:
	 
    Running MRIQC version 0.1.dev3288+g5f95281.d20230628:
      * BIDS dataset path: /data.
      * Output folder: /out.
      * Analysis levels: ['participant'].

230629-22:00:58,308 cli WARNING:
	 IMPORTANT: Anonymized quality metrics (IQMs) will be submitted to MRIQC's metrics repository. Submission of IQMs can be disabled using the ``--no-sub`` argument. Please visit https://mriqc.readthedocs.io/en/latest/dsa.html to revise MRIQC's Data Sharing Agreement.
230629-22:03:15,66 nipype.interface WARNING:
	 QC metrics upload failed to create an ID for the record uplOADED. rEsponse from server follows: {"_status": "ERR", "_error": {"code": 401, "message": "Please provide proper credentials"}}
230629-22:03:15,66 nipype.interface WARNING:
	 QC metrics failed to upload. Status 401: {"_status": "ERR", "_error": {"code": 401, "message": "Please provide proper credentials"}}
230629-22:03:18,603 cli WARNING:
	 IMPORTANT: Anonymized quality metrics (IQMs) wil

You are using MRIQC v0.1.dev3288+g5f95281.d20230628, and a newer version is available: 23.1.0.


230629-22:03:24,635 cli IMPORTANT:
	 
    Running MRIQC version 0.1.dev3288+g5f95281.d20230628:
      * BIDS dataset path: /data.
      * Output folder: /out.
      * Analysis levels: ['participant'].

230629-22:03:30,953 cli WARNING:
	 IMPORTANT: Anonymized quality metrics (IQMs) will be submitted to MRIQC's metrics repository. Submission of IQMs can be disabled using the ``--no-sub`` argument. Please visit https://mriqc.readthedocs.io/en/latest/dsa.html to revise MRIQC's Data Sharing Agreement.
230629-22:05:47,708 nipype.interface WARNING:
	 QC metrics upload failed to create an ID for the record uplOADED. rEsponse from server follows: {"_status": "ERR", "_error": {"code": 401, "message": "Please provide proper credentials"}}
230629-22:05:47,708 nipype.interface WARNING:
	 QC metrics failed to upload. Status 401: {"_status": "ERR", "_error": {"code": 401, "message": "Please provide proper credentials"}}
230629-22:05:51,248 cli WARNING:
	 IMPORTANT: Anonymized quality metrics (IQMs) w

You are using MRIQC v0.1.dev3288+g5f95281.d20230628, and a newer version is available: 23.1.0.
Traceback (most recent call last):
  File "/opt/conda/bin/mriqc", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.9/site-packages/mriqc/cli/run.py", line 206, in main
    dataframe, out_tsv = generate_tsv(output_dir, mod)
  File "/opt/conda/lib/python3.9/site-packages/mriqc/utils/misc.py", line 181, in generate_tsv
    jsonfiles = list(output_dir.glob("sub-*/**/%s/sub-*_%s.json" % (IMTYPES[mod], mod)))
KeyError: 'dwi'
